In [1]:
%pip install investpy

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import wrds
# import data_read
import pull_WRDS_call_reports, pull_treasuries_data
import pull_mbb_data
import data_preprocessing, compute_treasury_changes
# import calc_functions
# import Calc_table_statistic
import helper_functions

warnings.filterwarnings("ignore")

In [3]:
# idrssd_to_lei = pd.read_parquet('../manual data/idrssd_to_lei.parquet')
# lei_legalevents = pd.read_parquet('../manual data/lei_legalevents.parquet')
# lei_main = pd.read_parquet('../manual data/lei_main.parquet')
# lei_successorentity = pd.read_parquet('../manual data/lei_successorentity.parquet')
# wrds_bank_crsp_link = pd.read_parquet('../manual data/wrds_bank_crsp_link.parquet')
# wrds_call_research = pd.read_parquet('../data/manual/wrds_call_research.parquet')
wrds_call_research = pull_WRDS_call_reports.load_wrds_call_research()
# wrds_struct_rel_ultimate = pd.read_parquet('../manual data/wrds_struct_rel_ultimate.parquet')

In [4]:
# wrds_bank_crsp_link['rssd9001'] = wrds_bank_crsp_link['rssd9001'].astype('int')
wrds_call_research['rssd9001'] = wrds_call_research['rssd9001'].astype('int')

From Table A1, you need:
	1.	Total Assets (assets)
	2.	Cash Holdings (cash)
	3.	Securities (Total, Treasury, RMBS, CMBS, ABS, Other) (securities, possibly other breakdowns)
	4.	Loans (Total, Real Estate, Commercial, Consumer, etc.)
	5.	Reverse Repo and Fed Funds Sold (reverse_repo, fedfundsrate)
	6.	Categorization into Bank Size Groups
	•	Small banks: Assets < $1.384B
	•	Large banks (non-GSIB): Assets ≥ $1.384B
	•	GSIB: Systemically important banks (may require separate classification logic)

Step 3: Categorize Banks by Size: Use the assets column to classify banks

Step 4: Compute Median & Standard Deviation for Each Group

In [5]:
import pandas as pd
import numpy as np
import datetime

# Step 1: Select data for Q1 2022 explicitly
report_date = datetime.date(2022, 3, 31)
df_date = wrds_call_research[wrds_call_research['date'] == pd.Timestamp(report_date)].copy()

# Verify unique banks at reporting date
num_banks = df_date['rssd9001'].nunique()
print(f"Number of banks as of {report_date}: {num_banks}")

Number of banks as of 2022-03-31: 4913


In [6]:
# small, large(non gsib), gsib
# Categorize banks based on total asset size and GSIB status
small_bank_threshold = 1_384_000  # Assets in thousands (1.384B)

# First classify banks into Small vs Large
df_date["bank_category"] = np.where(
    df_date["assets"] < small_bank_threshold, "Small", "Large (non-GSIB)"
)

# Define GSIB banks explicitly by RSSD IDs
gsib_list = [
    852218, 480228, 476810, 413208, # JP Morgan, Bank of America, Citigroup, HSBC
    2980209, 2182786, 541101, 655839, 1015560, 229913, # Barclays, Goldman Sachs, BNY Mellon, etc.
    1456501, 722777, 35301, 925411, 497404, 3212149, # Morgan Stanley, Santander, State Street, etc.
    451965 # Wells Fargo
]

# Within large banks, classify GSIB banks separately
df_date.loc[
    (df_date["bank_category"] == "Large (non-GSIB)") & 
    (df_date["rssd9001"].isin(gsib_list)), 
    "bank_category"
] = "GSIB"

# Verify categories
print(df_date["bank_category"].value_counts())

bank_category
Small               4064
Large (non-GSIB)     833
GSIB                  16
Name: count, dtype: int64


In [7]:
# Explicitly calculation: other_security & other_re_loan & loanstonondep
df_date['other_security'] = df_date['securities'].fillna(0) - (
    df_date[['treasurysec', 'residentialmbsat', 'commercialmbsat', 'absassets']].fillna(0).sum(axis=1)
)

df_date['other_re_loan'] = df_date['reloans'].fillna(0) - (
    df_date[['loans1to4fam', 'realoansnonres']].fillna(0).sum(axis=1)
)

# Loan to Non-Depository (loanstonondep)
df_date['loanstonondep'] = df_date[['otherciloans', 'otherbankacceptances']].fillna(0).sum(axis=1)

In [8]:
# Define all asset-related columns clearly and explicitly
asset_cols = {
    'cash': 'Cash',
    'securities': 'Security',
    'treasurysec': 'Treasury',
    'residentialmbsat': 'RMBS',
    'commercialmbsat': 'CMBS',
    'absassets': 'ABS',
    'other_security': 'Other Security',
    'loansnet': 'Total Loan',
    'reloans': 'Real Estate Loan',
    'loans1to4fam': 'Residential Mortgage',
    'realoansnonres': 'Commercial Mortgage',
    'other_re_loan': 'Other Real Estate Loan',
    'agloans': 'Agricultural Loan',
    'ciloans': 'Commercial & Industrial Loan',
    'persloans': 'Consumer Loan',
    'loanstonondep': 'Loan to Non-Depository',
    'fedfundsrepoasset': 'Fed Funds Sold',
    'fedfundsrepoliab': 'Reverse Repo' 
}


In [9]:
# Temporarily fill any missing columns with zeros (for safety)
for col in ['loans1to4fam', 'realoansnonres', 'loanstonondep', 'fedfundsrepoliab']:
    if col not in df_date.columns:
        df_date[col] = 0

# Compute percentages of each asset category relative to total assets
df_date['assets'] = df_date['assets'].replace({0: np.nan})

for col in asset_cols.keys():
    df_date[f'{col}_pct'] = (df_date[col].fillna(0) / df_date['assets']) * 100

In [10]:
# Calculate Aggregate separately (sum percentages explicitly)
aggregate_assets_sum = df_date['assets'].sum()

aggregate_pct_dict = {
    f'{col}_pct': (df_date[col].sum() / aggregate_assets_sum) * 100
    for col in asset_cols.keys()
}

aggregate_df = pd.DataFrame(aggregate_pct_dict, index=['Aggregate'])

In [11]:
# Compute median and std for other bank categories
grouped = df_date.groupby('bank_category')

non_agg_summary = grouped[[f'{col}_pct' for col in asset_cols]].agg(['median', 'std'])

# Add total assets (in billions) and number of banks explicitly
non_agg_summary[('Total Asset ($B)', '')] = grouped['assets'].sum() / 1e6
non_agg_summary[('Number of Banks', '')] = grouped['rssd9001'].nunique()

# Flatten multi-level column index
non_agg_summary.columns = [' '.join(col).strip() for col in non_agg_summary.columns.values]

In [12]:
non_agg_summary2 = grouped[[f'{col}' for col in asset_cols]].agg(['sum'])
non_agg_summary2.columns = [' '.join(col).strip() for col in non_agg_summary2.columns.values]

In [13]:
non_agg_summary2

,cash sum,securities sum,treasurysec sum,residentialmbsat sum,commercialmbsat sum,absassets sum,other_security sum,loansnet sum,reloans sum,loans1to4fam sum,realoansnonres sum,other_re_loan sum,agloans sum,ciloans sum,persloans sum,loanstonondep sum,fedfundsrepoasset sum,fedfundsrepoliab sum
bank_category,,,,,,,,,,,,,,,,,,
GSIB,2.053435e+09,3.152952e+09,1.118273e+09,1.336627e+09,155512854.0,56320806.0,486218497.0,4.224498e+09,1.457615e+09,9.345195e+08,2.633472e+08,259748688.0,7517035.0,9.560306e+08,784583731.0,9.560306e+08,593263864.0,215932258.0
Large (non-GSIB),2.124927e+09,2.696509e+09,2.973664e+08,1.388990e+09,348546264.0,66509866.0,595096930.0,6.814094e+09,3.156543e+09,1.322191e+09,1.163678e+09,670674732.0,24959316.0,1.663309e+09,955206428.0,1.662436e+09,579681476.0,665493024.0
Small,1.865693e+08,3.317781e+08,3.874296e+07,1.038388e+08,15527491.0,4085911.0,169583009.0,8.336472e+08,6.398355e+08,2.285223e+08,2.471286e+08,164184728.0,32014806.0,1.199595e+08,35361667.0,1.199595e+08,17396564.0,10442818.0


In [14]:
# Rename columns clearly
rename_dict = {}
for key, val in asset_cols.items():
    rename_dict[f'{key}_pct median'] = f'{val} Median'
    rename_dict[f'{key}_pct std'] = f'{val} Std'

non_agg_summary.rename(columns=rename_dict, inplace=True)

In [15]:
# Explicit calculation of Aggregate row (sum percentages)
aggregate_data = {f'{val} Median': (df_date[col].sum() / aggregate_assets_sum) * 100 for col, val in asset_cols.items()}
aggregate_summary = pd.DataFrame(aggregate_pct_dict, index=['Aggregate'])

# Add total asset sum and bank count explicitly for aggregate
aggregate_extra = pd.DataFrame({
    'Total Asset ($B)': aggregate_assets_sum / 1e6,  # convert thousands to billions
    'Number of Banks': df_date['rssd9001'].nunique()
}, index=['Aggregate'])

In [16]:
aggregate_summary

,cash_pct,securities_pct,treasurysec_pct,residentialmbsat_pct,commercialmbsat_pct,absassets_pct,other_security_pct,loansnet_pct,reloans_pct,loans1to4fam_pct,realoansnonres_pct,other_re_loan_pct,agloans_pct,ciloans_pct,persloans_pct,loanstonondep_pct,fedfundsrepoasset_pct,fedfundsrepoliab_pct
Aggregate,15.83261,22.420778,5.275378,10.263088,1.88466,0.460356,4.537296,43.063343,19.057446,9.01451,6.072541,3.970395,0.233924,9.936068,6.438884,9.932902,4.317644,3.235011


In [17]:
# Step 10: Clearly define Total Asset and Number of Banks
aggregate_assets_sum = df_date['assets'].sum()
aggregate_extra = pd.DataFrame({
    'Total Asset ($B)': [aggregate_assets_sum / 1e6],  # Convert from thousands to billions
    'Number of Banks': df_date['rssd9001'].nunique()
}, index=['Aggregate'])

# Clearly define aggregate percentage DataFrame
aggregate_pct_dict = {
    f'{col}_pct': (df_date[col].sum() / aggregate_assets_sum) * 100
    for col in asset_cols.keys()
}
aggregate_pct_df = pd.DataFrame(aggregate_pct_dict, index=['Aggregate'])

# Create std rows as NaNs explicitly for aggregate (since it's sum, no std)
aggregate_std_df = pd.DataFrame({f'{col}_pct': np.nan for col in asset_cols.keys()}, index=['Aggregate'])

# Rename columns clearly (match non_agg_summary columns)
aggregate_pct_df.columns = [f'{asset_cols[col.split("_pct")[0]]} Median' for col in aggregate_pct_df.columns]

# Combine median and NaN std explicitly
aggregate_summary_final = pd.concat([aggregate_extra, aggregate_pct_df], axis=1)
for col in aggregate_summary_final.columns:
    if 'Median' in col:
        aggregate_summary_final[col.replace('Median', 'Std')] = np.nan  # explicitly NaN for Aggregate std

# Ensure columns order matches non-aggregate exactly
aggregate_summary_final = aggregate_summary_final[non_agg_summary.columns]

# Step 11: Combine aggregate and non-aggregate clearly
final_summary = pd.concat([aggregate_summary_final, non_agg_summary])

# Replace infinite values with NaN and round clearly
final_summary.replace([np.inf, -np.inf], np.nan, inplace=True)
final_summary = final_summary.round(2)

# After you obtain final_summary (before transposing):

# Reorder columns clearly to put "Total Asset" and "Number of Banks" first
first_cols = ['Total Asset ($B)', 'Number of Banks']
other_cols = [col for col in final_summary.columns if col not in first_cols]

final_summary = final_summary[first_cols + other_cols]

# Now transpose explicitly
final_summary['Number of Banks'] = final_summary['Number of Banks'].astype(int)
final_table = final_summary.T

In [18]:
final_summary['Number of Banks']

Aggregate           4913
GSIB                  16
Large (non-GSIB)     833
Small               4064
Name: Number of Banks, dtype: int64

In [19]:
final_table_styled = (
    final_table.style
    .format(precision=2, na_rep='')
    .set_properties(**{
        'text-align': 'center',
        'font-size': '14px',
        'padding': '8px'
    })
    .set_table_styles([
        {'selector': 'th',
         'props': [('background-color', 'white'),
                   ('color', 'black'),
                   ('text-align', 'center'),
                   ('font-size', '15px'),
                   ('border-bottom', '2px solid black'),
                   ('padding', '10px')]},
        {'selector': 'td',
         'props': [('background-color', 'white'),
                   ('color', 'black')]}
    ])
)

# Display styled table explicitly
display(final_table_styled)

,Aggregate,GSIB,Large (non-GSIB),Small
Total Asset ($B),27569.25,11701.52,14428.40,1439.33
Number of Banks,4913.00,16.00,833.00,4064.00
Cash Median,15.83,18.02,8.61,10.90
Cash Std,,21.77,16.35,13.71
Security Median,22.42,20.79,17.60,21.91
Security Std,,16.53,15.04,17.29
Treasury Median,5.28,5.25,0.22,0.32
Treasury Std,,6.20,4.42,6.29
RMBS Median,10.26,5.48,5.58,3.87
RMBS Std,,7.35,8.65,8.82


In [20]:
final_table = final_table.rename(index=lambda x: x.replace("Commercial & Industrial", "C/I"))
final_table = final_table.rename(index=lambda x: x.replace("Total Asset ($B)", "Total Asset (Billions)"))

In [21]:
helper_functions.save_dataframe_as_latex_table_1(final_table, filename="replicated_table_1A.tex", caption="Replicated Table 1A: Bank Asset Composition", label="tab:replicated1A")

LaTeX table saved as replicated_table_1A.tex


In [22]:
def dataframe_to_image(df, filename="dataframe.png"):
    fig, ax = plt.subplots(figsize=(len(df.columns) * 1.2, len(df) * 0.5))  # Adjust size dynamically
    ax.axis('tight')
    ax.axis('off')
    
    table = pd.plotting.table(ax, df, loc='center', cellLoc='center', colWidths=[0.2]*len(df.columns))
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1.2, 1.2)  # Adjust scale
    
    plt.savefig(filename, bbox_inches='tight', dpi=300)
    plt.close()


# dataframe_to_image(final_table, "table_a1.png")


In [23]:
final_table

,Aggregate,GSIB,Large (non-GSIB),Small
Total Asset (Billions),27569.25,11701.52,14428.40,1439.33
Number of Banks,4913.00,16.00,833.00,4064.00
Cash Median,15.83,18.02,8.61,10.90
Cash Std,NaN,21.77,16.35,13.71
Security Median,22.42,20.79,17.60,21.91
Security Std,NaN,16.53,15.04,17.29
Treasury Median,5.28,5.25,0.22,0.32
Treasury Std,NaN,6.20,4.42,6.29
RMBS Median,10.26,5.48,5.58,3.87
RMBS Std,NaN,7.35,8.65,8.82


In [24]:
non_agg_summary2.rename(columns=lambda x: x.replace(" sum", ""), inplace=True)
non_agg_summary2

,cash,securities,treasurysec,residentialmbsat,commercialmbsat,absassets,other_security,loansnet,reloans,loans1to4fam,realoansnonres,other_re_loan,agloans,ciloans,persloans,loanstonondep,fedfundsrepoasset,fedfundsrepoliab
bank_category,,,,,,,,,,,,,,,,,,
GSIB,2.053435e+09,3.152952e+09,1.118273e+09,1.336627e+09,155512854.0,56320806.0,486218497.0,4.224498e+09,1.457615e+09,9.345195e+08,2.633472e+08,259748688.0,7517035.0,9.560306e+08,784583731.0,9.560306e+08,593263864.0,215932258.0
Large (non-GSIB),2.124927e+09,2.696509e+09,2.973664e+08,1.388990e+09,348546264.0,66509866.0,595096930.0,6.814094e+09,3.156543e+09,1.322191e+09,1.163678e+09,670674732.0,24959316.0,1.663309e+09,955206428.0,1.662436e+09,579681476.0,665493024.0
Small,1.865693e+08,3.317781e+08,3.874296e+07,1.038388e+08,15527491.0,4085911.0,169583009.0,8.336472e+08,6.398355e+08,2.285223e+08,2.471286e+08,164184728.0,32014806.0,1.199595e+08,35361667.0,1.199595e+08,17396564.0,10442818.0


In [25]:
aggregate_sums = non_agg_summary2.sum()
total_assets_sum = aggregate_sums[["cash", "securities", "reloans", "ciloans", "other_security"]]# .sum()
total_liabilities_sum = aggregate_sums[["loansnet", "loanstonondep", "fedfundsrepoasset", "fedfundsrepoliab"]]# .sum()

# # Compute proportions
# asset_aggregates = {
#     category: (aggregate_sums[category] / total_assets_sum) * 100
#     for category in ["cash", "securities", "reloans", "ciloans", "other_security"]
# }

# liability_aggregates = {
#     category: (aggregate_sums[category] / total_liabilities_sum) * 100
#     for category in ["loansnet", "loanstonondep", "fedfundsrepoasset", "fedfundsrepoliab"]
# }

# # Convert to DataFrame for display
# aggregate_df = pd.DataFrame([asset_aggregates, liability_aggregates], index=["Assets (%)", "Liabilities (%)"])

In [26]:
total_assets_sum, total_liabilities_sum

(cash              4.364931e+09
 securities        6.181240e+09
 reloans           5.253994e+09
 ciloans           2.739299e+09
 other_security    1.250898e+09
 dtype: float64,
 loansnet             1.187224e+10
 loanstonondep        2.738426e+09
 fedfundsrepoasset    1.190342e+09
 fedfundsrepoliab     8.918681e+08
 dtype: float64)

In [27]:
import importlib
importlib.reload(helper_functions)
helper_functions.save_stacked_bar_chart(total_assets_sum, total_liabilities_sum, "assets_liabilities_chart.png")